# NSW Conservation and Sensitive Lists

## List Identifiers
* Conservation list: __[dr650](https://lists.ala.org.au/speciesListItem/list/dr650)__ (and [dr650 in test](https://lists-test.ala.org.au/speciesListItem/list/dr650))
* Collection: __[dr650](https://collections.ala.org.au/public/show/dr650)__
### Source Data

*  __[Home Page](https://data.bionet.nsw.gov.au)__
*  __[API/Data Info](https://data.bionet.nsw.gov.au/biosvcapp/odata/SpeciesNames)__
*  __[Data Information and file downloads](https://www.environment.nsw.gov.au/research-and-publications/publications-search/bionet-sensitive-species-list)__


**Metadata Description**
Classification codes under the Threatened Species Conservation Act 1995: * Presumed Extinct * Critically Endangered * Endangered * Vulnerable * Endangered Population Learn more about the classifications used by the NSW Office of Environment & Heritage Machine-readable downloads can be obtained by searching NSW BioNet, the website for the Atlas of NSW Wildlife, and selecting: 1. Which species or group? = "all species", 2. Legal status = "Select records that fall under one or more categories" then choose "Threatened in NSW" 3. What area? = "Entire area" 4. Period of records? = "All records"

**Metadata URL**
https://data.bionet.nsw.gov.au


Import required libraries
Retrieve dataset from API
Set Project directory

In [8]:
import pandas as pd
import urllib.request, json
import os
import sys
import datetime

projectDir = "/Users/oco115/PycharmProjects/authoritative-lists/"
statusDir = projectDir + "source-data/status-codes/"
# projectDir = "/Users/new330/IdeaProjects/authoritative-lists/"
sys.path.append(os.path.abspath(projectDir + "source-code/includes"))
monthStr = datetime.datetime.now().strftime('%Y%m')
import list_functions as lf


In [ ]:
# %%script echo skipping # comment this line to download the files and save locally

with urllib.request.urlopen("https://data.bionet.nsw.gov.au/biosvcapp/odata/SpeciesNames") as url:
    data = json.loads(url.read().decode())
data = pd.json_normalize(data, record_path =['value'])

In [ ]:
## Process Conservation
cdf= data[(data['stateConservation'] !='Not Listed') & (data['isCurrent'] == 'true')]
cdf['stateConservation'].unique()

In [7]:
## Map statuses
# state = 'NSW'
# fname = statusDir + state + "-codes.csv"
# df = lf.map_status(state, fname, df)


KeyError: 'sourceStatus'

Extract records where speciesID = taxonID
Map columns to DWC terms
Map status from stateConservation column

In [10]:
conservationlist = cdf[(cdf['speciesID'] == cdf['taxonID'])]

In [11]:
conservationlist.columns

Index(['dcterms_rightsHolder', 'dcterms_rights', 'dcterms_language',
       'dcterms_type', 'dcterms_modified', 'dcterms_available', 'speciesID',
       'taxonRank', 'kingdomID', 'kingdom', 'classID', 'class', 'orderID',
       'order', 'familyID', 'family', 'sortOrder', 'genusID', 'genus',
       'parentSpeciesID', 'specificEpithet', 'infraspecificEpithet',
       'scientificNameAuthorship', 'scientificNameID', 'speciesCode_Synonym',
       'scientificName', 'scientificNameHTML', 'vernacularName',
       'otherVernacularNames', 'taxonID', 'currentScientificNameCode',
       'currentScientificName', 'currentVernacularName', 'isCurrent',
       'generalTypeID', 'generalType', 'establishmentMeans',
       'primaryGrowthForm', 'primaryGrowthFormGroup', 'secondaryGrowthForms',
       'secondaryGrowthFormGroups', 'stateConservation', 'protectedInNSW',
       'sensitivityClass', 'TSProfileID', 'countryConservation',
       'highThreatWeed', 'widelyCultivatedNativeSpecies', 'CAMBA', 'JAMBA',


In [12]:
conservationlist = conservationlist[
    ['scientificName', 'vernacularName', 'family', 'genus', 'stateConservation']]
conservationlist['status'] = conservationlist['stateConservation']
conservationlist = conservationlist.rename(columns={"stateConservation": "sourceStatus"})


## Write dataframe to CSV - UTF-8 encoding

In [ ]:
print("Writing to CSV")
dr = 'dr650'
conservationlist.to_csv(projectDir + 'current-lists/conservation-lists/NSW-' + dr + '-conservationlist.csv',encoding="UTF-8",index=False)
print('Finished writing NSW conservation list')

In [ ]:
### Change logs

In [ ]:
# NSW conservation
print('NSW conservation')
ltype = "C"
changeDir = "Monitoring/Change-logs/"
filename = "NSW-conservation.csv"
testdr = "dr650"
proddr = "dr650"
changelist = lf.get_changelist(testdr, proddr, ltype)
changelist.to_csv(projectDir + changeDir + monthStr + "-" + filename, encoding="UTF-8", index=False)
print('Finished NSW conservation')

In [ ]:
# Download historical lists from Production
prodListUrl = "https://lists.ala.org.au/ws/speciesListItems/" + proddr + "?max=10000&includeKVP=true"
prodList = lf.download_ala_specieslist(prodListUrl)  # save the prod list to the historical lists directory
prodList = lf.kvp_to_columns(prodList)
prodList.to_csv(projectDir + "historical-lists/conservation/" + filename, encoding="UTF-8", index=False)
print('Finished downloading conservation historical list')